In [468]:
import pandas as pd 
import numpy as np
import os

# Set up workspace path - go up one directory from scripts folder to project root
workspace_root = os.path.dirname(os.getcwd())

# Set the data directory path
data_dir = os.path.join(workspace_root, 'data')


In [469]:
# Read data
# Data comes from https://data360.worldbank.org/en/dataset/WB_WDI
csv_file_path = os.path.join(data_dir, 'WB_WDI_WIDEF.csv')

wdi_data = pd.read_csv(csv_file_path)
print(wdi_data.head())
print(wdi_data.columns)
print(wdi_data.shape)


  FREQ FREQ_LABEL REF_AREA REF_AREA_LABEL                 INDICATOR  \
0    A     Annual      ABW          Aruba     WB_WDI_AG_CON_FERT_ZS   
1    A     Annual      ABW          Aruba     WB_WDI_AG_LND_AGRI_K2   
2    A     Annual      ABW          Aruba     WB_WDI_AG_LND_AGRI_ZS   
3    A     Annual      ABW          Aruba     WB_WDI_AG_LND_ARBL_HA   
4    A     Annual      ABW          Aruba  WB_WDI_AG_LND_ARBL_HA_PC   

                                     INDICATOR_LABEL SEX SEX_LABEL AGE  \
0  Fertilizer consumption (kilograms per hectare ...  _T     Total  _T   
1                         Agricultural land (sq. km)  _T     Total  _T   
2                 Agricultural land (% of land area)  _T     Total  _T   
3                             Arable land (hectares)  _T     Total  _T   
4                  Arable land (hectares per person)  _T     Total  _T   

                               AGE_LABEL  ...         2015         2016  \
0  All age ranges or no breakdown by age  ...     3.1

In [471]:
# Evaluate the countries values
list_countries = wdi_data['REF_AREA_LABEL'].unique()
list_countries

# Remove goegraphic regions from REF_AREA_LABEL using the metadata information 
region_codes = [
    'AFE','ARB', 'CSS','EAR','EAP','EMU','EUU','FCS','IBD','IBT','IDB','IDX','IDA','LTE',
    'LCN','LAC','TLA','LDC','LMY','LIC','LMC','MEA','MNA','TMN','MIC','NAC','OED','PST',
    'PRE','SAS','TSA','SSF','SSA','TSS','UMC','WLD','HPC','AFW','TEA','HIC','EAS','SST',
    'OSS','CEB','ECS','ECA','TEC','PSS'
    'PSS'
]

wdi_data = wdi_data[~wdi_data['REF_AREA'].isin(region_codes)]
wdi_data.shape


(246178, 91)

In [472]:
# Save a dictionary of indicators using relative path
dictionary_indicators = wdi_data[['INDICATOR','INDICATOR_LABEL']].drop_duplicates()
output_path = os.path.join(data_dir, 'dictionary_indicators.csv')
dictionary_indicators.to_csv(output_path, index=False)


In [473]:
# First convert each year column to a row 
# identify year columns (1960-2024)
year_columns = [col for col in wdi_data.columns if col.isdigit() and len(col) == 4]

# Melt the data 
wdi_data_melted = pd.melt(
    wdi_data, 
    # Just the columns we want to keep 
    id_vars=['REF_AREA', 'REF_AREA_LABEL', 'INDICATOR', 'INDICATOR_LABEL'],
    value_vars=year_columns,
    var_name='YEAR',
    value_name='VALUE'
)

wdi_data_melted.head()


,REF_AREA,REF_AREA_LABEL,INDICATOR,INDICATOR_LABEL,YEAR,VALUE
0,ABW,Aruba,WB_WDI_AG_CON_FERT_ZS,Fertilizer consumption (kilograms per hectare ...,1960,NaN
1,ABW,Aruba,WB_WDI_AG_LND_AGRI_K2,Agricultural land (sq. km),1960,NaN
2,ABW,Aruba,WB_WDI_AG_LND_AGRI_ZS,Agricultural land (% of land area),1960,NaN
3,ABW,Aruba,WB_WDI_AG_LND_ARBL_HA,Arable land (hectares),1960,NaN
4,ABW,Aruba,WB_WDI_AG_LND_ARBL_HA_PC,Arable land (hectares per person),1960,NaN


In [474]:
# Now pivot to create wide format with indicators as columns
wdi_data_wide = wdi_data_melted.pivot_table(
    index=['REF_AREA', 'REF_AREA_LABEL', 'YEAR'], 
    columns='INDICATOR', 
    values='VALUE'
).reset_index()


#print(wdi_data_wide.head())
print(wdi_data_wide.shape)

(14170, 1447)


In [475]:
# Important variables
important_variables = wdi_data_wide[[

'REF_AREA', 'REF_AREA_LABEL',  'YEAR',

# Economic variables
'WB_WDI_NY_GDP_PCAP_KD', # GDP per capita (constant 2015 US$)
'WB_WDI_NY_GDP_MKTP_KD', #	GDP (constant 2015 US$)
'WB_WDI_NY_GDP_MKTP_KD_ZG', #	GDP growth (annual %)

# Population variables
'WB_WDI_SP_URB_TOTL_IN_ZS', # Urban population (% of total population)
'WB_WDI_SP_URB_GROW', # Urban population growth (annual %)
'WB_WDI_SE_PRM_ENRR',# School enrollment, primary (% gross)
'WB_WDI_SE_SEC_ENRR', #	School enrollment, secondary (% gross)
'WB_WDI_SE_TER_ENRR', # School enrollment, tertiary (% gross)
'WB_WDI_SP_POP_GROW',	# Population growth (annual %)
'WB_WDI_SP_POP_TOTL',	# Population, total
'WB_WDI_SE_ADT_LITR_ZS',	# Literacy rate, adult total (% of people ages 15 and above)
'WB_WDI_SE_ADT_1524_LT_ZS', # Literacy rate, youth total (% of people ages 15-24)

# Electricity variables
'WB_WDI_EG_ELC_RNWX_ZS', # Electricity production from renewable sources, excluding hydroelectric (% of total)
'WB_WDI_EG_ELC_ACCS_UR_ZS', # Access to electricity, urban (% of urban population)
'WB_WDI_EG_ELC_ACCS_ZS',	# Access to electricity (% of population)
'WB_WDI_EG_ELC_COAL_ZS',	# Electricity production from coal sources (% of total)
'WB_WDI_EG_ELC_FOSL_ZS',	# Electricity production from oil, gas and coal sources (% of total)
'WB_WDI_EG_ELC_HYRO_ZS',	# Electricity production from hydroelectric sources (% of total)
'WB_WDI_EG_ELC_NGAS_ZS',	# Electricity production from natural gas sources (% of total)
'WB_WDI_EG_ELC_NUCL_ZS',	# Electricity production from nuclear sources (% of total)
'WB_WDI_EG_ELC_PETR_ZS',	# Electricity production from oil sources (% of total)
'WB_WDI_EG_ELC_RNEW_ZS',	# Renewable electricity output (% of total electricity output)
'WB_WDI_EG_ELC_RNWX_KH',	# Electricity production from renewable sources, excluding hydroelectric (kWh)
'WB_WDI_EG_USE_PCAP_KG_OE', # Energy use (kg of oil equivalent per capita)
'WB_WDI_EG_FEC_RNEW_ZS', # Renewable energy consumption (% of total final energy consumption)
'WB_WDI_EG_USE_COMM_CL_ZS',	#Alternative and nuclear energy (% of total energy use)
'WB_WDI_EG_USE_COMM_FO_ZS',	#Fossil fuel energy consumption (% of total)
'WB_WDI_EG_USE_COMM_GD_PP_KD',	#Energy use (kg of oil equivalent) per $1,000 GDP (constant 2021 PPP)
'WB_WDI_EG_USE_CRNW_ZS',	#Combustible renewables and waste (% of total energy)
'WB_WDI_EG_USE_ELEC_KH_PC',	#Electric power consumption (kWh per capita)



# Human capital variables
'WB_WDI_HD_HCI_OVRL', # Human Capital Index (HCI) (scale 0-1)
'WB_WDI_SI_POV_GINI', #	Gini index


# Emissions variables
'WB_WDI_EN_GHG_ALL_PC_CE_AR5', # Total greenhouse gas emissions excluding LULUCF per capita (t CO2e/capita)
'WB_WDI_EN_GHG_CH4_AG_MT_CE_AR5', #	Methane (CH4) emissions from Agriculture (Mt CO2e)
'WB_WDI_EN_GHG_CH4_BU_MT_CE_AR5', #	Methane (CH4) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_FE_MT_CE_AR5', # Methane (CH4) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_IC_MT_CE_AR5', #	Methane (CH4) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_MT_CE_AR5',	#Methane (CH4) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_CH4_PI_MT_CE_AR5',	#Methane (CH4) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_TR_MT_CE_AR5',	#Methane (CH4) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_WA_MT_CE_AR5',	#Methane (CH4) emissions from Waste (Mt CO2e)
'WB_WDI_EN_GHG_CH4_ZG_AR5',	# Methane (CH4) emissions (total) excluding LULUCF (% change from 1990)
'WB_WDI_EN_GHG_CO2_BU_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_FE_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_IC_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_IP_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Industrial Processes (Mt CO2e)
'WB_WDI_EN_GHG_CO2_MT_CE_AR5',	#Carbon dioxide (CO2) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_CO2_PC_CE_AR5',	#Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)
'WB_WDI_EN_GHG_CO2_PI_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_RT_GDP_KD',	#Carbon intensity of GDP (kg CO2e per constant 2015 US$ of GDP)
'WB_WDI_EN_GHG_CO2_RT_GDP_PP_KD',	#Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP)
'WB_WDI_EN_GHG_CO2_TR_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_ZG_AR5',	#Carbon dioxide (CO2) emissions (total) excluding LULUCF (% change from 1990)
'WB_WDI_EN_GHG_N2O_AG_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Agriculture (Mt CO2e)
'WB_WDI_EN_GHG_N2O_BU_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_FE_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_IC_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_IP_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Industrial Processes (Mt CO2e)
'WB_WDI_EN_GHG_N2O_MT_CE_AR5',	#Nitrous oxide (N2O) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_N2O_PI_MT_CE_AR5',#	Nitrous oxide (N2O) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_TR_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_WA_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Waste (Mt CO2e)
'WB_WDI_EN_GHG_N2O_ZG_AR5',	#Nitrous oxide (N2O) emissions (total) excluding LULUCF (% change from 1990)
'WB_WDI_EN_GHG_TOT_ZG_AR5'	#Total greenhouse gas emissions excluding LULUCF (% change from 1990)

]]

important_variables.shape


(14170, 67)

In [476]:
# Get the most correlated variables 
# Exclude the columns that are not variables
data_corr = important_variables.loc[:, ~important_variables.columns.isin(['REF_AREA', 'REF_AREA_LABEL', 'YEAR'])]

# Correlation matrix 
data_corr = data_corr.loc[:, ~data_corr.columns.duplicated()]
correlations = data_corr.corr().abs().unstack().sort_values(ascending=False).drop_duplicates()
correlations

# Convert to DataFrame
correlations_df = pd.DataFrame({
     'Variable_1': correlations.index.get_level_values(0),
     'Variable_2': correlations.index.get_level_values(1), 
     'Correlation': correlations.values.round(3)
 })

# # Elimina correlaciones de variable consigo misma (por seguridad)
correlations_df = correlations_df[correlations_df['Variable_1'] != correlations_df['Variable_2']]



In [477]:
# Evaluate correlation between the variables of emissions

emissions_variables = [
'WB_WDI_EN_GHG_ALL_PC_CE_AR5', # Total greenhouse gas emissions excluding LULUCF per capita (t CO2e/capita)
'WB_WDI_EN_GHG_CH4_AG_MT_CE_AR5', #	Methane (CH4) emissions from Agriculture (Mt CO2e)
'WB_WDI_EN_GHG_CH4_BU_MT_CE_AR5', #	Methane (CH4) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_FE_MT_CE_AR5', # Methane (CH4) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_IC_MT_CE_AR5', #	Methane (CH4) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_MT_CE_AR5',	#Methane (CH4) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_CH4_PI_MT_CE_AR5',	#Methane (CH4) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_TR_MT_CE_AR5',	#Methane (CH4) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_WA_MT_CE_AR5',	#Methane (CH4) emissions from Waste (Mt CO2e)
'WB_WDI_EN_GHG_CH4_ZG_AR5',	# Methane (CH4) emissions (total) excluding LULUCF (% change from 1990)
'WB_WDI_EN_GHG_CO2_BU_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_FE_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_IC_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_IP_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Industrial Processes (Mt CO2e)
'WB_WDI_EN_GHG_CO2_MT_CE_AR5',	#Carbon dioxide (CO2) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_CO2_PC_CE_AR5',	#Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)
'WB_WDI_EN_GHG_CO2_PI_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_RT_GDP_KD',	#Carbon intensity of GDP (kg CO2e per constant 2015 US$ of GDP)
'WB_WDI_EN_GHG_CO2_RT_GDP_PP_KD',	#Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP)
'WB_WDI_EN_GHG_CO2_TR_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_ZG_AR5',	#Carbon dioxide (CO2) emissions (total) excluding LULUCF (% change from 1990)
'WB_WDI_EN_GHG_N2O_AG_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Agriculture (Mt CO2e)
'WB_WDI_EN_GHG_N2O_BU_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_FE_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_IC_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_IP_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Industrial Processes (Mt CO2e)
'WB_WDI_EN_GHG_N2O_MT_CE_AR5',	#Nitrous oxide (N2O) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_N2O_PI_MT_CE_AR5',#	Nitrous oxide (N2O) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_TR_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_WA_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Waste (Mt CO2e)
'WB_WDI_EN_GHG_N2O_ZG_AR5',	#Nitrous oxide (N2O) emissions (total) excluding LULUCF (% change from 1990)
'WB_WDI_EN_GHG_TOT_ZG_AR5'	#Total greenhouse gas emissions excluding LULUCF (% change from 1990)]]
]

print(correlations_df[correlations_df['Correlation'] >= 0.8].shape)

emissions_correlations = correlations_df[
    (correlations_df['Correlation'] >= 0.8) &
    ((correlations_df['Variable_1'].isin(emissions_variables)) & 
    (correlations_df['Variable_2'].isin(emissions_variables)))
]

print(emissions_correlations.shape)

# From 153 high correlations , 124 are among emissions variables


(153, 3)
(124, 3)


In [478]:
# What are that the high correlations between variables not related to emissions?
complement_high_emissions = correlations_df[
    ((correlations_df['Correlation'] >= 0.8) & 
    ~  ((correlations_df['Variable_1'].isin(emissions_variables)) & 
      (correlations_df['Variable_2'].isin(emissions_variables)))
      )
]

complement_high_emissions


,Variable_1,Variable_2,Correlation
10,WB_WDI_EG_ELC_HYRO_ZS,WB_WDI_EG_ELC_RNEW_ZS,0.963
11,WB_WDI_SE_ADT_1524_LT_ZS,WB_WDI_SE_ADT_LITR_ZS,0.958
13,WB_WDI_SP_POP_TOTL,WB_WDI_EN_GHG_N2O_WA_MT_CE_AR5,0.955
14,WB_WDI_EN_GHG_N2O_TR_MT_CE_AR5,WB_WDI_NY_GDP_MKTP_KD,0.945
17,WB_WDI_NY_GDP_MKTP_KD,WB_WDI_EN_GHG_CO2_TR_MT_CE_AR5,0.938
21,WB_WDI_EG_FEC_RNEW_ZS,WB_WDI_EG_USE_CRNW_ZS,0.934
30,WB_WDI_SP_POP_TOTL,WB_WDI_EN_GHG_CH4_AG_MT_CE_AR5,0.924
36,WB_WDI_EG_ELC_RNEW_ZS,WB_WDI_EG_ELC_FOSL_ZS,0.918
42,WB_WDI_EG_USE_COMM_FO_ZS,WB_WDI_EG_USE_CRNW_ZS,0.911
43,WB_WDI_EG_ELC_ACCS_ZS,WB_WDI_EG_ELC_ACCS_UR_ZS,0.911


In [479]:
# Evaluate the amount of missing values per column 
missing_values_total = important_variables.isnull().sum().reset_index(name='values').sort_values(by='values', ascending=False)
missing_values_total


,INDICATOR,values
33,WB_WDI_HD_HCI_OVRL,13569
13,WB_WDI_SE_ADT_LITR_ZS,13101
14,WB_WDI_SE_ADT_1524_LT_ZS,12948
34,WB_WDI_SI_POV_GINI,11819
30,WB_WDI_EG_USE_COMM_GD_PP_KD,9353
...,...,...
6,WB_WDI_SP_URB_TOTL_IN_ZS,130
12,WB_WDI_SP_POP_TOTL,30
1,REF_AREA_LABEL,0
2,YEAR,0


In [480]:
# Evaluate the amount of missing values per column for 2024
missing_values_2024= important_variables[important_variables['YEAR'] == '2024'].isnull().sum().reset_index(name='values').sort_values(by='values', ascending=False)

# The varibales that are missing among all the countries in 2024
missing_values_2024[missing_values_2024['values'] == 218]


,INDICATOR,values
33,WB_WDI_HD_HCI_OVRL,218
35,WB_WDI_EN_GHG_ALL_PC_CE_AR5,218
37,WB_WDI_EN_GHG_CH4_BU_MT_CE_AR5,218
38,WB_WDI_EN_GHG_CH4_FE_MT_CE_AR5,218
39,WB_WDI_EN_GHG_CH4_IC_MT_CE_AR5,218
40,WB_WDI_EN_GHG_CH4_MT_CE_AR5,218
41,WB_WDI_EN_GHG_CH4_PI_MT_CE_AR5,218
42,WB_WDI_EN_GHG_CH4_TR_MT_CE_AR5,218
43,WB_WDI_EN_GHG_CH4_WA_MT_CE_AR5,218
44,WB_WDI_EN_GHG_CH4_ZG_AR5,218


In [481]:
# How many countries I have in the data? 
len(important_variables['REF_AREA_LABEL'].unique())


218

In [482]:
# What are the countries with highest missing values in 2024? 
data_2024 = important_variables[important_variables['YEAR'] == '2024']
missing_by_country = data_2024.groupby('REF_AREA_LABEL').apply(lambda x: x.isnull().sum())
missing_by_country_sorted = missing_by_country.sum(axis=1).sort_values(ascending=False).reset_index(name='missing_values')

print(missing_by_country_sorted.describe())
missing_by_country_sorted[missing_by_country_sorted['missing_values'] > 57]


       missing_values
count      218.000000
mean        57.458716
std          1.111582
min         56.000000
25%         57.000000
50%         57.000000
75%         57.000000
max         62.000000


/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/4231906112.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_country = data_2024.groupby('REF_AREA_LABEL').apply(lambda x: x.isnull().sum())


,REF_AREA_LABEL,missing_values
0,St. Martin (French part),62
1,Afghanistan,60
2,French Polynesia,60
3,Bhutan,60
4,Greenland,60
5,Tonga,60
6,British Virgin Islands,60
7,Guam,60
8,Palau,60
9,Syrian Arab Republic,60


In [483]:
# There are so many NULL columns, what about 2020?
# What are the countries with highest missing values in 2024? 
data_2024 = important_variables[important_variables['YEAR'] == '2020']
missing_by_country = data_2024.groupby('REF_AREA_LABEL').apply(lambda x: x.isnull().sum())
missing_by_country_sorted = missing_by_country.sum(axis=1).sort_values(ascending=False).reset_index(name='missing_values')

print(missing_by_country_sorted.describe())
# Columns look better here

       missing_values
count      218.000000
mean         8.926606
std         11.765141
min          0.000000
25%          2.000000
50%          4.000000
75%         10.000000
max         55.000000


/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/284238620.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_country = data_2024.groupby('REF_AREA_LABEL').apply(lambda x: x.isnull().sum())


In [ ]:
# Cleanning the data

In [484]:
# Kepp only data from 1995 to 2024
important_variables['YEAR'] = pd.to_numeric(important_variables['YEAR'])
print(important_variables['YEAR'].describe())
important_variables = important_variables[important_variables['YEAR'] >= 1995]


count    14170.000000
mean      1992.000000
std         18.762325
min       1960.000000
25%       1976.000000
50%       1992.000000
75%       2008.000000
max       2024.000000
Name: YEAR, dtype: float64


/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/3572178775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_variables['YEAR'] = pd.to_numeric(important_variables['YEAR'])


In [485]:
# Percentage of missing values per column
missing_perc = (important_variables.isna().mean()*100).reset_index(name='missing_percentage').sort_values(by='missing_percentage', ascending=False)
print(missing_perc['missing_percentage'].describe())
missing_perc.head(10)


count    67.000000
mean     18.742754
std      19.142843
min       0.000000
25%       9.541284
50%      13.532110
75%      18.134557
max      90.810398
Name: missing_percentage, dtype: float64


,INDICATOR,missing_percentage
33,WB_WDI_HD_HCI_OVRL,90.810398
13,WB_WDI_SE_ADT_LITR_ZS,86.376147
14,WB_WDI_SE_ADT_1524_LT_ZS,84.036697
34,WB_WDI_SI_POV_GINI,69.296636
10,WB_WDI_SE_TER_ENRR,44.724771
9,WB_WDI_SE_SEC_ENRR,39.143731
30,WB_WDI_EG_USE_COMM_GD_PP_KD,37.140673
31,WB_WDI_EG_USE_CRNW_ZS,37.003058
32,WB_WDI_EG_USE_ELEC_KH_PC,35.963303
28,WB_WDI_EG_USE_COMM_CL_ZS,35.932722


In [486]:
# Remove columns with >50% missing
remove_columns = ['WB_WDI_SI_POV_GINI','WB_WDI_SE_ADT_1524_LT_ZS','WB_WDI_SE_ADT_LITR_ZS',
 'WB_WDI_HD_HCI_OVRL']

important_variables = important_variables.drop(columns=remove_columns)


In [487]:
# Drop countries with almost no data

# Total number of values per country (excluding the country column)
total_values = important_variables.drop(columns=['REF_AREA_LABEL']).groupby(important_variables['REF_AREA_LABEL']).size() * important_variables.drop(columns=['REF_AREA_LABEL']).shape[1]
    
# Count total missing values per country
na_count = important_variables.groupby('REF_AREA_LABEL').apply(lambda x: x.drop(columns=['REF_AREA_LABEL']).isna().sum().sum())
    
# Combine into a summary DataFrame
summary = pd.DataFrame({
        "na_count": na_count,
        "total_values": total_values
    })

summary["na_percent"] = (summary["na_count"] / summary["total_values"]) * 100
summary = summary.sort_values(by="na_percent", ascending=False).reset_index()
summary['na_percent'].describe()

# Based on distrubution
countires_remove = summary[summary['na_percent'] >=17]['REF_AREA_LABEL'].tolist()
important_variables = important_variables[~important_variables['REF_AREA_LABEL'].isin(countires_remove)]
len(important_variables['REF_AREA_LABEL'].unique())

# We kept 159 countries

/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/3040928623.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  na_count = important_variables.groupby('REF_AREA_LABEL').apply(lambda x: x.drop(columns=['REF_AREA_LABEL']).isna().sum().sum())


159

In [489]:
# Linear interpolation
# Only interpolate forward. As I do not have data previous I prefer to not guess. 
important_variables = (
    important_variables
    .sort_values(by=['REF_AREA_LABEL', 'YEAR'])          #sort by country and year
    .groupby('REF_AREA_LABEL')                           # group by country
    .apply(lambda group: group.interpolate(method='linear', limit_direction='both'))  # interpolate within each group
    .reset_index(drop=True)                              
)


important_variables.isna().sum().reset_index(name='missing_values').sort_values(by='missing_values', ascending=False).head(20)


/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/351355353.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.interpolate(method='linear', limit_direction='both'))  # interpolate within each group
/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/351355353.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.interpolate(method='linear', limit_direction='both'))  # interpolate within each group
/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_26455/351355353.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda

,INDICATOR,missing_values
29,WB_WDI_EG_USE_CRNW_ZS,480
26,WB_WDI_EG_USE_COMM_CL_ZS,480
30,WB_WDI_EG_USE_ELEC_KH_PC,480
28,WB_WDI_EG_USE_COMM_GD_PP_KD,240
49,WB_WDI_EN_GHG_CO2_RT_GDP_PP_KD,120
24,WB_WDI_EG_USE_PCAP_KG_OE,120
27,WB_WDI_EG_USE_COMM_FO_ZS,120
42,WB_WDI_EN_GHG_CO2_FE_MT_CE_AR5,90
54,WB_WDI_EN_GHG_N2O_FE_MT_CE_AR5,60
10,WB_WDI_SE_TER_ENRR,60


In [490]:
# Save the data
important_variables.to_csv(os.path.join(data_dir, 'important_variables.csv'), index=False)